#MoneyGram Lend v1.2

The App is available [here](https://moneygram-lend.anvil.app)

In [1]:
# Install Uplink to run web app
!pip install anvil-uplink

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


In [2]:
#Import Dependencies
import pickle
from google.colab import drive
import anvil.server

In [3]:
#Connect to Anvil Server 
anvil.server.connect("VZCQ27KKQZFQ4LACSW6MW2SI-RHNXEOV2BAZECHG4")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment" as SERVER


In [4]:
#Mount The Drive to access saved model
drive.mount('/content/gdrive')
#Define path to retrieve Pickle File
DATA_PATH = "/content/gdrive/My Drive"
#Define Model name
model = "/finalized_model.sav"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [5]:
#Open the Pickle File
filename = open(DATA_PATH+ model,'rb')

In [6]:
# Load the model from Google Drive Disk
loaded_model = pickle.load(filename)

In [12]:
#Call the Anvil Server
@anvil.server.callable

#Predict Loan Function
def predict_loan(home, age, income, amount):

  #Format home data
  person_home_ownership_MORTGAGE = 0
  person_home_ownership_OTHER = 0
  person_home_ownership_OWN = 0
  person_home_ownership_RENT = 0

  #Conditional 
  if home == "mortgage":
    person_home_ownership_MORTGAGE = 1

  if home == "other":
    person_home_ownership_OTHER = 1

  if home == "own":
    person_home_ownership_OWN = 1

  if home == "rent":
    person_home_ownership_RENT = 1

  #Call The ML model and return a Sigmoid Classification
  classification = loaded_model.predict([[person_home_ownership_MORTGAGE, person_home_ownership_OTHER, person_home_ownership_OWN, person_home_ownership_RENT, age, income, amount]])

  #Variable Rate based on customer risk 
  #If non risky then interest is 3%
  if classification == 0:
    response = f"Your loan will be {amount * 1.03}"
    
  #If risky then interest is 4% with initial deposit
  if classification == 1:
    response = f"Your loan will be ${amount * 1.04} and you will need to make an initial deposit of ${amount * 0.3}"

  #Return The Data
  return response

In [8]:
# #Test Prediction
# loaded_model.predict([[1.0,0.0,0.0,0.0,25,9600,5500]])

In [9]:
#Keep Colab session running 
anvil.server.wait_forever()

KeyboardInterrupt: ignored